In [3]:
import sys
sys.path.append('..')

# Import Libraries
from Utils.Accuracy_measures import topk_accuracy
from Utils.TinyImageNet_loader import get_tinyimagenet_dataloaders
from Utils.Num_parameter import count_parameters
from Models.vit_base import VisionTransformer

import torchvision.transforms as transforms
from torch import nn
from torch import optim

import time
import torch
import os

In [4]:
n_epoch = 5
print(f'Training for {len(range(n_epoch))} epochs\n')
for epoch in range(0+1,n_epoch+1):
    print(epoch)

Training for 5 epochs

1
2
3
4
5
